In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/Legend13")
import torch 
import torch.nn as nn
!pip install torchinfo
from torchinfo import summary

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!jupyter nbconvert --to markdown /content/drive/MyDrive/Legend13/Self_Implement/psm_ResNeXt.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/Legend13/Self_Implement/psm_ResNeXt.ipynb to markdown
[NbConvertApp] Writing 20060 bytes to /content/drive/MyDrive/Legend13/Self_Implement/psm_ResNeXt.md


## **Split-Transform-Merge Strategy**

<br/>

- Grouped convolution performs a set of transformations that consist of a low-dimensional embedding and aggregation of output. 
- All transformations are in same topology, which distinguishes ResNeXt from Inception-ResNet (Inception v4) that also involves branching and concatenating in residual function.

&emsp;&emsp;&emsp;**Figure 1. : Original bottleneck block in ResNet vs ResNeXt with caridnality = 32 with roughly same complexity**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/50bc4d51-21c5-4844-8a37-2ff86536e0f0" width="600">

<br/>

### **Aggregated Transformation**

<br/>

- $\large y = x + \sum_{i=1}^{C} T_i(x)$, &emsp; where $\large T_i(x)$ has same topology for all $\large i$

<br/>

## **Equivalent Building Blocks for ResNeXt**

<br/>

&emsp;&emsp;&emsp;**Figure 3. : Equivalent blcoks for ResNeXt (<span style="color:blue">Figure 3-(c)</span> is the final implementation of gouped convolution)**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/6ec35f31-6ee4-4311-9280-74cad1ccd068" width="900">


<br/>

## **Basic Building Block (Bottleneck)** 

<br/>

- Example of basic bottleneck building block in Conv2 stage of ResNeXt architecture

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/7f9d4a29-aafc-4ff4-977e-8f976e6b0ba0" width="200">

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/36388eba-896e-4904-9cf0-2b7451c263f1" width="600">

<br/>

- grouped convolution layer is a **wider and sparsely-connected** version of original ResNeXt in Figure 1. (left)

<br/>

## **Comparison with Other Neural Architectures**

<br/>

### **1. Cardinality vs Width**

<br/>

&emsp;&emsp;&emsp;**Table 2. : Trade-Off between cardinality and width of block (d) with preserved complexity**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/59de951b-1f17-479e-b8ae-442725d8c07a" width="500">

- Example of how to determine appropriate trade-off combination of cardinality and bottleneck width while maintaining the complexity 

    - 1) Original ResNet bottleneck block has 256 x 64 + 3 x 3 x 64 x 64 + 64 x 256 ≈ 70k parameters
    - 2) With bottleneck width = d, parameters = C · (256 · d + 3 · 3 · d · d + d · 256) 
    - find the combination of C and d that gives roughly identical complexity.

<br/>

&emsp;&emsp;&emsp;**Figure 5. Comparison of train and validation accuracy between ResNet 50, 101 (1 x 64d) and ResNeXt-50, 101 (32 x 4d)**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/d58a61a3-9812-41c0-bdeb-d958cc243568" width="900">

&emsp;&emsp;&emsp;**Table 3. : Results of increasing cardinality at the expense of bottleneck width (d) (Experiments on ImageNet-1K)**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/4b75d1e6-a4ca-4a95-a022-d173fda95793" width="450">

<br/>

- for both 50 and 101 layer networks, every ResNeXt with cardinality increasing from 2 to 32 outperforms its ResNet couterpart. 

- Because the improvement from trade-off becomes saturated as cardinality increases, the paper adopted the bottleneck width no smaller than 4d. 


<br/>

### **2. Cardinality vs Depth/Width with Increased Complexity**

<br/>

&emsp;&emsp;&emsp;**Table 4. : Comparison of deeper/wider/increased cardinality networks with complexity doubled to ResNet-101's**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/a84ca011-8686-4dd7-9272-1703262cc028" width="500">

<br/>

- highlighted part indicates where the complexity is increased (baseline : 1x complexity reference)

- ResNeXt-101 models with increased cardinality (2 x 64d, 64 x 4d) record lower top-1 and top-5 error rate compared to both deeper and wider ResNet.

- What's remarkable here is that ResNeXt with 1x complexity (32 x 4d) performs better than ResNet-200 and wider ResNet-101 even with 50% complexity. 

<br/>

&emsp;&emsp;&emsp;**Figure 7. Effects of Increasing Complexity (Parameters) with Cardinality vs Width (on CIFAR-10)**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/1397d0af-80c7-4a59-97c9-3289a6416885" width="500">

<br/>

- It's always better to increase cardinality instead of transformation width to reduce the test error rate 

<br/>

### **3. ResNet vs Inception Net vs ResNeXt**

<br/>

&emsp;&emsp;&emsp;**Table 5. experimented on ImageNet-1K**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/8ff68e2f-a1cd-48e9-84e9-22e7ea28e60e" width="530">


<br/>

### **4. With or Without Residual (Skip) Connections**

<br/>

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/6ee04d46-eaff-4165-ad84-e78cb93e53fe" width="420">


- Introducing grouped convolution method shows consistent improvement in the performance of networks with or without skip connections.

- Residual connections are known to relieve non-convexities existing in the loss surface, resulting in easier and faster optimization. 

- Considering all these, residual connections are helpful
for optimization process, whereas aggregated transformations achieve
stronger representations where networks learn to capture more representationally important patterns or features of the input images.  

## **ResNeXt Architecture**

<br/>

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/a03bfa29-4755-4461-9bf9-3f928365b7f8" width="600">

<br/>

- Performs downsampling with stride 2 3x3 grouped convolution for every first block at each conv stage. 



## **Implementation with PyTorch**

In [ ]:
class Bottleneck(nn.Module):
    expansion = 2
    def __init__(self, in_channels, out_channels, cardinality=32, stride=1, projection=False):
        super().__init__()

        self.residual = nn.Sequential(nn.Conv2d(in_channels, out_channels, 1, bias=False),
                                    nn.BatchNorm2d(out_channels),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(out_channels, out_channels, 3, stride=stride, padding=1, groups=cardinality, bias=False),
                                    nn.BatchNorm2d(out_channels),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(out_channels, out_channels*self.expansion, 1, bias=False),
                                    nn.BatchNorm2d(out_channels*self.expansion))
        if projection:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels*self.expansion, 1, stride=stride),
                                          nn.BatchNorm2d(out_channels*self.expansion))
        else:
            self.shortcut = nn.Identity()

        self.relu = nn.ReLU()

    def forward(self, x):
        residual = self.residual(x)
        shortcut = self.shortcut(x)

        return self.relu(residual + shortcut)


class ResNeXt(nn.Module):
    def __init__(self, in_channels, block, expansion, cardinality, block_repeats, num_classes, zero_init_residual=True):
        super().__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, 64, 7, stride=2, padding=3),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True))
        self.conv2_pool = nn.MaxPool2d(3, stride=2, padding=1)

        self.expansion = expansion
        out_channels, self.conv2_blocks = self.stack_blocks(block, 64, block_repeats[0], cardinality, 2)
        out_channels, self.conv3_blocks = self.stack_blocks(block, out_channels, block_repeats[1], cardinality, 2)
        out_channels, self.conv4_blocks = self.stack_blocks(block, out_channels, block_repeats[2], cardinality, 2)
        out_channels, self.conv5_blocks = self.stack_blocks(block, out_channels, block_repeats[3], cardinality, 2)

        self.gap = nn.AdaptiveAvgPool2d((1,1))   # 1x1x2048
        self.classifier = nn.Linear(out_channels, 1000)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            # zero-initialize the last BN in each residual branch -> set the start of residual branch as zero (identity mapping)
            # improves model by 0.2~0.3%p (https://arxiv.org/abs/1706.02677)
            elif isinstance(m, block):    
                nn.init.constant_(m.residual[-1].weight, 0)

    def stack_blocks(self, block, in_channel, block_repeat, cardinality, stride):
        stacked = []

        c, repeats = block_repeat
        for _ in range(repeats):
            if stride == 2 or in_channel != c*self.expansion:
                stacked += [block(in_channel, c, cardinality, stride, True)]
                in_channel = c*self.expansion
                stride = 1
            else:
                stacked += [block(in_channel, c, cardinality)]
                in_channel = c*self.expansion

        return c*self.expansion, nn.Sequential(*stacked)  

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2_pool(x)
        x = self.conv2_blocks(x)
        x = self.conv3_blocks(x)
        x = self.conv4_blocks(x)
        x = self.conv5_blocks(x)
        x = self.gap(x)
        x = torch.flatten(x, start_dim=1)
        out = self.classifier(x)

        return out

## **Model Summary**


In [ ]:
block_repeats = [(128,3), (256,4), (512,6), (1024,3)]
expansion = 2
cardinality = 32

model = ResNeXt(3, Bottleneck, expansion, cardinality, block_repeats, 1000)
summary(model, input_size=(2, 3, 224, 224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
ResNeXt                                  [2, 1000]                 --
├─Sequential: 1-1                        [2, 64, 112, 112]         --
│    └─Conv2d: 2-1                       [2, 64, 112, 112]         9,472
│    └─BatchNorm2d: 2-2                  [2, 64, 112, 112]         128
│    └─ReLU: 2-3                         [2, 64, 112, 112]         --
├─MaxPool2d: 1-2                         [2, 64, 56, 56]           --
├─Sequential: 1-3                        [2, 256, 28, 28]          --
│    └─Bottleneck: 2-4                   [2, 256, 28, 28]          --
│    │    └─Sequential: 3-1              [2, 256, 28, 28]          46,592
│    │    └─Sequential: 3-2              [2, 256, 28, 28]          17,152
│    │    └─ReLU: 3-3                    [2, 256, 28, 28]          --
│    └─Bottleneck: 2-5                   [2, 256, 28, 28]          --
│    │    └─Sequential: 3-4              [2, 256, 28, 28]          71,168

In [ ]:
x = torch.randn(2,3,224,224)
out = model(x)
print(out.shape)
out

torch.Size([2, 1000])


tensor([[ 1.3636,  1.0331, -0.1438,  ...,  0.6048,  0.3152,  0.3505],
        [ 1.3330,  0.8973, -0.1342,  ...,  0.5798,  0.0394,  0.3434]],
       grad_fn=<AddmmBackward0>)

## **Other types of Convolution**

<br/>

- In computer vision, it is important to retain enough contextual information to make accurate classification for a given input image. This can be done by increasing the kernel size of receptive field or concatenating multiple convolution layers (5x5 = 3x3 + 3x3). These approaches, However, dramatically increase the computational complexities. To tackle this issue, researchers have developed a variety of other convolution methods that aim to reduce the computations while preserving the spatial information. Here are a few examples of these convolution methods.

<br/>


### **Depthwise Pointwise Convolution**

<br/>

- Traditional convolution layers perform spatial and channel-wise convolution at the same time, which means a single output feature requries a separate set of filters with complete shape. 

- Depthwise pointwise convolution, on the other hand, splits the convolution operation into two separate stages: depthwise convolution and pointwise convolution. 

&emsp;&emsp;&emsp; Depthwise Convolution & Pointwise Convolution

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/15328448-2431-4f00-80ab-013f316a2ba3" width="400">

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/e33db26f-34eb-4dd3-a5f8-8f6e7dd11497" width="400"> 


1. **Depthwise Convolution** : 
    - Idential to the grouped convolution with cardinality equal to total channel size. 
    - Only performs spatial convolution where each channel of the input requires a single filter and concatenates the resultant feature maps.
    - Each feature map contains spatial representation within one chanenl. 
    - This reduces the network complexities, as each filter is only responsible for convolving with its corresponding input channel.   

2. **Pointwise Convolution** : 
    - Applying convolution with size 1, stride 1, merging the output of depthwise convolution into a desired number of feature maps. 
    - Creates new representations by linear combination of all channels with learned weights. 

- One can improve model capacity by increasing parameters using the saved resources from depthwise convolution

<br/>

### **Depthwise Separable Convolution**

<br/>

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/0fe43c30-9765-47c9-8198-68d96241ea23" width="600"> 

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/421bbbfb-1e45-42f0-8993-b4ac0833b506" width="420"> 

<br/>

- Same as the former one, but last combining step with pointwise convolution is replaced with separable convoluiton where nxn convolution is splited into 1xn convolution (horizontal) and nx1 convolution (vertical)

